In [1]:
import pyrosetta 
import rosetta 
from sys import argv
import os 
os.environ[ "SLURM_ARRAY_TASK_ID" ] = "96" # for testing! 

In [2]:
with open( '../../mutant_list.txt' ) as fn:
    mutant_list = fn.read().split()
    mutant_name = mutant_list[ int( os.environ[ 'SLURM_ARRAY_TASK_ID' ] ) - 1 ] 

In [3]:
fmt = dict( zip( 'ANDRCQEGHILKMPFSTWYV', [
    'ALA','ASN','ASP','ARG','CYS','GLN','GLU',
    'GLY','HIS','ILE','LEU','LYS','MET','PRO','PHE','SER',
    'THR','TRP','TYR','VAL' ] ) ) 

In [4]:
# flags from command line 
with open( 'input_files/flags' ) as fn:
    flags = fn.read().replace( '\n', ' ' )
pyrosetta.init( ''.join( flags ) ) 

# extra residue (pNPG) params 
ligand_params = pyrosetta.Vector1( [ 'input_files/pNPG.params' ] )
new_res_set = pyrosetta.generate_nonstandard_residue_set( ligand_params )

# construct pose 
p = pyrosetta.Pose()
pyrosetta.pose_from_file( p, new_res_set, 'input_files/bglb.pdb' ) 

# construct score function 
scorefxn = pyrosetta.create_score_function( 'beta_cst' ) 

# add in enzyme design constraints 
add_cst = rosetta.protocols.enzdes.AddOrRemoveMatchCsts()
add_cst.cstfile( 'input_files/pNPG.enzdes.cst' ) 
add_cst.set_cst_action( rosetta.protocols.enzdes.CstAction.ADD_NEW )
add_cst.apply( p ) 

# mutate the residue 
target = int( mutant_name[ 1:-1 ] )
new_res = fmt[ mutant_name[ -1 ] ] 
mut = rosetta.protocols.simple_moves.MutateResidue( target, new_res )
mut.apply( p ) 

Found rosetta database at: /usr/local/bin/anaconda3/lib/python3.5/site-packages/pyrosetta-4.0-py3.5.egg/database; using it....
PyRosetta-4 2016 [Rosetta 2016 unknown:7862c3a14bb5e8d6b2a4c3e0a497795bc06e9e4b 2016-12-13 14:39:52 -0500] retrieved from: git@github.com:RosettaCommons/main.git
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.



In [8]:
# set up pack task 
tf = rosetta.core.pack.task.TaskFactory()
around = rosetta.protocols.toolbox.task_operations.DesignAroundOperation()
around.include_residue( 446 ) #ligand 
around.include_residue( target ) 
around.repack_shell( 15 ) # let's grid search this  
around.resnums_allow_design( False )
around.allow_design( False )
tf.push_back( around ) 
pt = tf.create_task_and_apply_taskoperations(p)
print( pt )

# repack and minimize 
repack = rosetta.protocols.enzdes.EnzRepackMinimize()
repack.set_scorefxn_repack( scorefxn )
repack.set_scorefxn_minimize( scorefxn )
repack.set_min_bb( True )
repack.set_min_lig( True )  
repack.set_min_rb( True ) 
repack.set_min_sc( True )
repack.task_factory( tf ) # adds packer task 

# monte carlo 
parsed = rosetta.protocols.simple_moves.GenericMonteCarloMover()
parsed.set_mover( repack )
parsed.set_maxtrials( 10 )
parsed.set_scorefxn( scorefxn )
parsed.apply( p ) 

# output PDB, probably for local testing only 
p.dump_pdb( '/Users/alex/Desktop/output_files/{}.pdb'.format( mutant_name) ) 

#Packer_Task

resid	pack?	design?	allowed_aas
1	FALSE	FALSE	
2	FALSE	FALSE	
3	FALSE	FALSE	
4	FALSE	FALSE	
5	FALSE	FALSE	
6	FALSE	FALSE	
7	FALSE	FALSE	
8	FALSE	FALSE	
9	FALSE	FALSE	
10	FALSE	FALSE	
11	FALSE	FALSE	
12	FALSE	FALSE	
13	FALSE	FALSE	
14	TRUE	FALSE	SER
15	TRUE	FALSE	THR
16	TRUE	FALSE	SER
17	TRUE	FALSE	SER
18	TRUE	FALSE	TYR
19	TRUE	FALSE	GLN
20	TRUE	FALSE	ILE
21	FALSE	FALSE	
22	FALSE	FALSE	
23	FALSE	FALSE	
24	FALSE	FALSE	
25	FALSE	FALSE	
26	FALSE	FALSE	
27	FALSE	FALSE	
28	FALSE	FALSE	
29	FALSE	FALSE	
30	FALSE	FALSE	
31	FALSE	FALSE	
32	FALSE	FALSE	
33	FALSE	FALSE	
34	TRUE	FALSE	TRP
35	FALSE	FALSE	
36	FALSE	FALSE	
37	TRUE	FALSE	PHE
38	FALSE	FALSE	
39	FALSE	FALSE	
40	FALSE	FALSE	
41	FALSE	FALSE	
42	FALSE	FALSE	
43	FALSE	FALSE	
44	TRUE	FALSE	VAL
45	TRUE	FALSE	ILE
46	FALSE	FALSE	
47	FALSE	FALSE	
48	FALSE	FALSE	
49	FALSE	FALSE	
50	FALSE	FALSE	
51	FALSE	FALSE	
52	FALSE	FALSE	
53	FALSE	FALSE	
54	FALSE	FALSE	
55	FALSE	FALSE	
56	FALSE	FALSE	
57	FALSE	FALSE	
58	FALSE	FALSE	
59	FALSE	FALS

True

In [13]:
report = rosetta.protocols.features.ReportToDB()
help( report ) 
#report.apply( p ) 

Help on ReportToDB in module rosetta.protocols.features object:

class ReportToDB(rosetta.protocols.moves.Mover)
 |  Method resolution order:
 |      ReportToDB
 |      rosetta.protocols.moves.Mover
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...) from builtins.PyCapsule
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: handle) -> None
 |      
 |      2. __init__(self: handle, name: str) -> None
 |      
 |      3. __init__(self: handle, arg0: rosetta.utility.sql_database.session, arg1: str, arg2: str) -> None
 |      
 |      doc
 |      
 |      4. __init__(self: handle, arg0: rosetta.utility.sql_database.session, arg1: str, arg2: str, arg3: bool) -> None
 |      
 |      doc
 |      
 |      5. __init__(self: handle, db_session: rosetta.utility.sql_database.session, batch_name: str, batch_description: str, use_transactions: bool, cache_size: int) -> None
 |      
 |      6. __init__(self: handle, arg0: str

In [6]:
! ls /Users/alex/Desktop/output_files/

M261E.pdb


In [5]:
! cat features.py

import pyrosetta 
import rosetta 

# input 
with open( 'input_files/features.flags' ) as fn:
    flags = fn.read().replace( '\n', ' ' )

pyrosetta.init( ''.join( flags ) ) 

ligand_params = pyrosetta.Vector1( [ 'input_files/pNPG.params' ] )
new_res_set = pyrosetta.generate_nonstandard_residue_set( ligand_params )

p = pyrosetta.Pose()
pyrosetta.pose_from_file( p, new_res_set, 'output_files/H178K.pdb' ) 
scorefxn = pyrosetta.create_score_function( 'beta_cst' ) 

report = rosetta.protocols.features.ReportToDB()
report.add_features_reporter( rosetta.protocols.features.JobDataFeatures() ) 
#report.add_features_reporter( rosetta.protocols.features.StructureScoresFeatures( scorefxn ) ) 
report.apply( p ) 



In [ ]:
all_select = rosetta.core.select.residue_selector.TrueResidueSelector()
my_vector = all_select.apply( p ) 

In [ ]:
report = rosetta.protocols.features.ReportToDB()
report.add_features_reporter( rosetta.protocols.features.ResidueFeatures() )
report.report_features( p, 9, my_vector )